# Project: **[Surprise](https://github.com/NicolasHug/Surprise)**

[Latest commit as of writing (on master)](https://github.com/NicolasHug/Surprise/commit/5eb9017bcbe19e16ae251b2598eb8b768368023b)

Surprise is a Python library maintained by one of the core members of sklearn (popular Python ML library), Nicolas Hub, for recommender systems. Recommender systems often use matrix factorization to find latent features that can relate a specific user to a set of items. One common example of an application is Netflix, where subscribers get recommended movies based on not only what they've already watched buy also what people with similar consumption habits watch.

Currently the repo has 3.9k stars and uses Travis CI for its [automated testing](https://travis-ci.org/github/NicolasHug/Surprise).

I was looking at a few of the top recommender systems on GitHub and I really liked this one because it uses [Cython](https://cython.org/). Cython is a compiler that adds some extra syntax on top of Python to make it compile to optimized C code. I've never used it before but the promise is that you can import Cython modules (.pyx extension) just as you would normal Python (.py) modules.

## Why factorize?

The problem with recommender systems is that the user-item matrices are very sparse. Generally you have user rows ($n$) and item columns ($m$). One row vector would be that user's preferences towards items $0..m$. Since generally most users do not interact with all items in a system (i.e. no one buys everything on Amazon or watches all Netflix movies), the resulting $n \times m$ matrix is often impractical.

The idea is to create some $k$ number of new features that we can use in estimating a factorization of the sparse $n \times m$ matrix. If we factorize, we end up with 2 smaller matrices, $n \times k$ and $k \times m$ that will multiply to approximate the full $n \times m$ matrix.

## Method: SVD

Several methods for factorizing in recommender systems are used in practice: SVD (singular value decomposition), SVD++, and NMF (non-negative matrix factorization). Those are all implemented by this library.

I will be looking at the implementation of SVD, since it was famously invented(?) by Simon Funk to win the [Netflix Prize](https://en.wikipedia.org/wiki/Netflix_Prize) back in 2009. Check out Funk's [blog post](https://sifter.org/~simon/journal/20061211.html) describing it.

[Click here to read the docs for the `SVD` class in Surprise.](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD)

The `SVD` class extends `AlgoBase` which has a few methods, but `fit` and `predict` are the most common. This is mirrors the API for sklearn, unsurprisingly.

The singular value decomposition is calculated using stochastic gradient descent (SGD).

Here is the sample code copied verbatim from the repository's README:

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# Use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /Users/evan/.surprise_data/ml-100k
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9323  0.9355  0.9425  0.9331  0.9386  0.9364  0.0038  
MAE (testset)     0.7339  0.7377  0.7416  0.7352  0.7398  0.7376  0.0029  
Fit time          4.56    4.54    4.53    4.68    4.56    4.58    0.05    
Test time         0.19    0.16    0.12    0.17    0.13    0.15    0.03    


{'test_rmse': array([0.93225485, 0.93546637, 0.94253587, 0.93312308, 0.93857135]),
 'test_mae': array([0.73387052, 0.73767968, 0.74164673, 0.7352077 , 0.7397516 ]),
 'fit_time': (4.559683799743652,
  4.543395042419434,
  4.533591032028198,
  4.675574064254761,
  4.564362049102783),
 'test_time': (0.1928119659423828,
  0.16218090057373047,
  0.12331604957580566,
  0.16736984252929688,
  0.1266157627105713)}

As you can see it's pretty simple. We can use other matrix factorization algorithms by importing `SVDpp` (SVD++) or `NMF` (non-negative matrix factorization) like this: `from surprise import SVD, SVDpp, NMF`. The cross validation just helps show that we are not overfitting and that we are making accurate predictions. It eliminates some training data each iteration and will test against the rest.

## How SVD is estimated

Again, the goal is to find latent featurse that can be used to relate user preferences to item qualities. In Funk's blog post, this looks like:

```ratingsMatrix[user][movie] = sum (userFeature[f][user] * movieFeature[f][movie]) for f from 1 to 40```

a.k.a. each rating is presumed to be the dot product between a user-feature vector and a feature-item vector.

The SVD in Surprise is computed iteratively using SGD. Our goal is to find values for the following vectors ([permalink](https://github.com/NicolasHug/Surprise/blob/5eb9017bcbe19e16ae251b2598eb8b768368023b/surprise/prediction_algorithms/matrix_factorization.pyx#L193)):

```python
# user biases
cdef np.ndarray[np.double_t] bu
# item biases
cdef np.ndarray[np.double_t] bi
# user factors
cdef np.ndarray[np.double_t, ndim=2] pu
# item factors
cdef np.ndarray[np.double_t, ndim=2] qi
```

I don't really understand the user and items biases, but basically each user and item gets some scalar weight associated with it.

The user and item factors are the matrices that we can dot columns and rows of together in order to find item ratings for a particular user.


Here is the tight loop that estimates them ([permalink](https://github.com/NicolasHug/Surprise/blob/5eb9017bcbe19e16ae251b2598eb8b768368023b/surprise/prediction_algorithms/matrix_factorization.pyx#L228)):

```python
for current_epoch in range(self.n_epochs):
    if self.verbose:
        print("Processing epoch {}".format(current_epoch))
    for u, i, r in trainset.all_ratings():

        # compute current error
        dot = 0  # <q_i, p_u>
        for f in range(self.n_factors):
            dot += qi[i, f] * pu[u, f]
        err = r - (global_mean + bu[u] + bi[i] + dot)

        # update biases
        if self.biased:
            bu[u] += lr_bu * (err - reg_bu * bu[u])
            bi[i] += lr_bi * (err - reg_bi * bi[i])

        # update factors
        for f in range(self.n_factors):
            puf = pu[u, f]
            qif = qi[i, f]
            pu[u, f] += lr_pu * (err * qif - reg_pu * puf)
            qi[i, f] += lr_qi * (err * puf - reg_qi * qif)
```

First, we have a parameter that defines "epochs", basically the number of iterations that SGD will run before it says "ok we are good enough, here is the SVD". Then in each epoch, we loop through the entire User-Item ratings matrix (`for u, i, r in trainset.all_ratings()`). `Trainset#all_ratings` is a method that generates tuples for each of the ratings, probably for convenience since looping through that matrix is quite common in the project.

From here we compute the dot product (have to do it manually as a performance reason to do with Cython I think) between user factors and item factors in order to find the error compared to the actual rating (`r`).

Then we update biases (again I am not entirely sure how the biases play a role in the calculation).

When we finally update the factors (the guesstimated user-factor and item-factor matrices), we adjust the factor weights based on the error and its direction. `lr_pu` and `lr_qi` are the learning rates for user factors and item factors, respectively.

### Questions I have about the method

I don't know how the user and item biases come into play. In Simon Funk's blog ([same link as above](https://sifter.org/~simon/journal/20061211.html)) he refers to them as `userValue[]` and `itemValue[]`, if that helps answer it. 

## Other questions

In Funk's blog also he says that the diagonal matrix of singular values will not appear with his method of SVD, but it is trivial to extract from the feature matrices.

> If you're wondering where the diagonal scaling matrix is, it gets arbitrarily rolled in to the two side matrices, but could be trivially extracted if needed.

How would this be done exactly?